In [2]:
# written on 11/15/23 by @caramnix 
import pandas as pd
import numpy as np

In [11]:
df= pd.read_csv("/Users/caranix/Library/CloudStorage/OneDrive-TheOhioStateUniversity/2023-2024/Fall/CSPAN/116th_transcript_data/116th_date_joined.csv")


In [245]:
def find_string(data):
    data= data.lower()
    data = data.replace('-', ' ')      
    return ("one minute" in data)

In [18]:
df["one_minute_bin"] = df["text"].apply(find_string)

In [34]:
one_minute_df = df[df["one_minute_bin"] ==True]

In [83]:
one_minute_df['index_col']= one_minute_df.index

/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_77054/376822450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_minute_df['index_col']= one_minute_df.index


In [130]:
one_minute_df.head()

,url,time,name,party-state,text,congress,i,date,one_minute_bin,index,index_col
1,https://www.c-span.org/video/?456931-5&action=...,00:00:00,NaN,NaN,rr for one minute.,116,13,"JANUARY 17, 2019",True,1,1
15,https://www.c-span.org/video/?456931-5&action=...,00:09:17,NaN,NaN,the chair will now entertain requests for one-...,116,13,"JANUARY 17, 2019",True,15,15
20,https://www.c-span.org/video/?456931-5&action=...,00:11:12,NaN,NaN,"without objection, the gentleman is recognize...",116,13,"JANUARY 17, 2019",True,20,20
23,https://www.c-span.org/video/?456931-5&action=...,00:12:07,NaN,NaN,"madam speaker, i ask unanimous consent to addr...",116,13,"JANUARY 17, 2019",True,23,23
24,https://www.c-span.org/video/?456931-5&action=...,00:12:12,NaN,NaN,"without objection, the gentlewoman is recogni...",116,13,"JANUARY 17, 2019",True,24,24


In [ ]:
# mentions one minute, then the NEXT one is the one-minute speech 

# mentions one minute, then the next statement is "without objection, the gentlewoman is recogni..." and it's the NEXT one with the speech 

# idea: everytime one-minute mentioned, take next 5 lines.
# if the next line has "without objection", go to the next line! 
# then if the next line has a named speaker, this is the one minute speech-
# record "start time" / time stamp 
# also check to see if the next (2) lines have same speaker,
# lump them all into one  if so, since these are the same speech. 
# also take time stamp from *next* line after speech done, add as row "end time" 



In [146]:
# preininitalize columns of df 
#colnames= ["url", "start_time", "end_time", "name", "party_state", "text", 'congress', "date"]
#new_df = pd.DataFrame(columns = colnames)

In [147]:
#for i in range(0, len(one_minute_df)): 
#    index= int(one_minute_df.iloc[i].index_col)
#    
#    if pd.isna(df.iloc[index+1]["party-state"]) == False: #if name of next line is not NA... 
#        url = df.iloc[index+1].url 
#        start_time =  df.iloc[index+1].time
#        name = df.iloc[index+1]["name"]  
#        party_state =  df.iloc[index+1]["party-state"]
#        congress = 116 
#        date= df.iloc[index+1].date
        
        #this should be 1 min speech
#        if df.iloc[index+1]["name"] == df.iloc[index+2]["name"]: #if name matches for next two, combine 
#            text_of_speech= df.iloc[index+1].text + df.iloc[index+2].text
#            end_time = df.iloc[index+3].time 
#
#        else: 
#            text_of_speech= df.iloc[index+1].text
#            end_time = df.iloc[index+2].time
#    
#    
#        list_i = [url, start_time, end_time, name, party_state,text_of_speech, congress, date]
#        new_df.loc[len(new_df)] = list_i
#    else: #if name is na 
#       pass 

In [168]:
# preininitalize columns of df 
colnames= ["url", "start_time", "end_time", "name", "party_state", "text", 'congress', "date"]
new_df = pd.DataFrame(columns = colnames)

In [169]:
ct=0 
forgotten_indicies= []
for i in range(0, len(one_minute_df)): 
    index= int(one_minute_df.iloc[i].index_col)
    
    if pd.isna(df.iloc[index+1]["name"]) == False: #if name of next line is not NA... 
        url = df.iloc[index+1].url 
        start_time =  df.iloc[index+1].time
        name = df.iloc[index+1]["name"]  
        party_state =  df.iloc[index+1]["party-state"]
        congress = 116 
        date= df.iloc[index+1].date
        
        #this should be 1 min speech
        if df.iloc[index+1]["name"] == df.iloc[index+2]["name"]: #if name matches for next two, combine 
            text_of_speech= df.iloc[index+1].text + df.iloc[index+2].text
            end_time = df.iloc[index+3].time 
            ct+=1

        else: 
            text_of_speech= df.iloc[index+1].text
            end_time = df.iloc[index+2].time
    
    
        list_i = [url, start_time, end_time, name, party_state,text_of_speech, congress, date]
        new_df.loc[len(new_df)] = list_i
    else: #if name of next is na 
        
        # if it says is recognized and doesn't say false, i.e. won't be caught otherwise 
        if ("is recognized" in df.iloc[index+1].text) and (df.iloc[index+1].one_minute_bin == False): 
            
            #if the name isn't NA for the next one 
            if pd.isna(df.iloc[index+2]["name"]) == False:
                url = df.iloc[index+2].url 
                start_time =  df.iloc[index+2].time
                name = df.iloc[index+2]["name"]  
                party_state =  df.iloc[index+2]["party-state"]
                congress = 116 
                date= df.iloc[index+2].date
                
                #this should be 1 min speech
                if df.iloc[index+2]["name"] == df.iloc[index+3]["name"]: #if name matches for next two, combine 
                    text_of_speech= df.iloc[index+2].text + df.iloc[index+3].text
                    end_time = df.iloc[index+4].time 
                    ct+=1
                else: 
                    text_of_speech= df.iloc[index+2].text
                    end_time = df.iloc[index+3].time
            list_i = [url, start_time, end_time, name, party_state,text_of_speech, congress, date]
            new_df.loc[len(new_df)] = list_i
            
        else: 
            #also appends ones who get caught the next go around, i.e. because one_minute_bin is True 
            forgotten_indicies.append(i)
                
            
    
    #mini_df= df.iloc[index: index+5]


In [160]:
ct # 1,500 times we combined 

1523

In [180]:
#forgotten_indicies[0:5]

In [181]:
#one_minute_df.iloc[5]

In [182]:
#df[one_minute_df.iloc[5].index_col: one_minute_df.iloc[5].index_col+5]

In [170]:
new_df

,url,start_time,end_time,name,party_state,text,congress,date
0,https://www.c-span.org/video/?456931-5&action=...,00:00:05,00:03:19,Steny Hoyer,D-MD,"steny hoyer, d-mdmr. speaker, from time to tim...",116,"JANUARY 17, 2019"
1,https://www.c-span.org/video/?456931-5&action=...,00:09:29,00:11:03,Tony Cárdenas,D-CA,"tony cárdenas, d-cathank you very much, madam ...",116,"JANUARY 17, 2019"
2,https://www.c-span.org/video/?456931-5&action=...,00:11:15,00:12:04,Ralph Norman,R-SC,"ralph norman, r-scmadam speaker, i rise today ...",116,"JANUARY 17, 2019"
3,https://www.c-span.org/video/?456931-5&action=...,00:12:15,00:13:22,Haley Stevens,D-MI,"haley stevens, d-mii rise today in support of ...",116,"JANUARY 17, 2019"
4,https://www.c-span.org/video/?456931-5&action=...,00:13:35,00:14:19,Scott Perry,R-PA,"scott perry, r-pathank you, madam speaker. i r...",116,"JANUARY 17, 2019"
...,...,...,...,...,...,...,...,...
6388,https://www.c-span.org/video/?466507-4&action=...,04:06:55,04:07:40,Ross Spano,R-FL,"ross spano, r-flthank you, mr. speaker. i rise...",116,"NOVEMBER 20, 2019"
6389,https://www.c-span.org/video/?466507-4&action=...,04:08:01,04:09:25,Madeleine Dean,D-PA,"madeleine dean, d-pathank you, mr. speaker. in...",116,"NOVEMBER 20, 2019"
6390,https://www.c-span.org/video/?466507-4&action=...,04:09:42,04:10:45,"Michael ""Mike"" Guest",R-MS,"michael ""mike"" guest, r-mson november 22, mr. ...",116,"NOVEMBER 20, 2019"
6391,https://www.c-span.org/video/?466507-4&action=...,04:14:21,04:15:19,Lizzie Fletcher,D-TX,"lizzie fletcher, d-txthank you, mr. speaker. t...",116,"NOVEMBER 20, 2019"


In [202]:
def remove_name_from_text(row):    
    name_party= row['name'].lower() + ", " + row["party_state"].lower() 
    without_name= row['text'].replace(name_party, " ")

    return(without_name.replace("show full text show less text", " "))
    

In [203]:
cleaned_text=[]
for i in range(0,len(new_df)): 
    cleaned_text.append(remove_name_from_text(new_df.iloc[i]))
    
    
    

In [205]:
new_df['cleaned_text']= cleaned_text

In [206]:
new_df

,url,start_time,end_time,name,party_state,text,congress,date,cleaned_text
0,https://www.c-span.org/video/?456931-5&action=...,00:00:05,00:03:19,Steny Hoyer,D-MD,"steny hoyer, d-mdmr. speaker, from time to tim...",116,"JANUARY 17, 2019","mr. speaker, from time to time, for whatever ..."
1,https://www.c-span.org/video/?456931-5&action=...,00:09:29,00:11:03,Tony Cárdenas,D-CA,"tony cárdenas, d-cathank you very much, madam ...",116,"JANUARY 17, 2019","thank you very much, madam speaker. i'm very ..."
2,https://www.c-span.org/video/?456931-5&action=...,00:11:15,00:12:04,Ralph Norman,R-SC,"ralph norman, r-scmadam speaker, i rise today ...",116,"JANUARY 17, 2019","madam speaker, i rise today in celebration of..."
3,https://www.c-span.org/video/?456931-5&action=...,00:12:15,00:13:22,Haley Stevens,D-MI,"haley stevens, d-mii rise today in support of ...",116,"JANUARY 17, 2019","i rise today in support of the nearly 15,000 ..."
4,https://www.c-span.org/video/?456931-5&action=...,00:13:35,00:14:19,Scott Perry,R-PA,"scott perry, r-pathank you, madam speaker. i r...",116,"JANUARY 17, 2019","thank you, madam speaker. i rise in recogniti..."
...,...,...,...,...,...,...,...,...,...
6388,https://www.c-span.org/video/?466507-4&action=...,04:06:55,04:07:40,Ross Spano,R-FL,"ross spano, r-flthank you, mr. speaker. i rise...",116,"NOVEMBER 20, 2019","thank you, mr. speaker. i rise today to congr..."
6389,https://www.c-span.org/video/?466507-4&action=...,04:08:01,04:09:25,Madeleine Dean,D-PA,"madeleine dean, d-pathank you, mr. speaker. in...",116,"NOVEMBER 20, 2019","thank you, mr. speaker. in our conversation a..."
6390,https://www.c-span.org/video/?466507-4&action=...,04:09:42,04:10:45,"Michael ""Mike"" Guest",R-MS,"michael ""mike"" guest, r-mson november 22, mr. ...",116,"NOVEMBER 20, 2019","on november 22, mr. lewis brenner, a world wa..."
6391,https://www.c-span.org/video/?466507-4&action=...,04:14:21,04:15:19,Lizzie Fletcher,D-TX,"lizzie fletcher, d-txthank you, mr. speaker. t...",116,"NOVEMBER 20, 2019","thank you, mr. speaker. today on transgender ..."


In [208]:
new_df.to_csv("/Users/caranix/Library/CloudStorage/OneDrive-TheOhioStateUniversity/2023-2024/Fall/CSPAN/116th_transcript_data/cleaned_transcript_data_116th.csv")




In [ ]:
# 115th 

In [216]:
df= pd.read_csv("/Users/caranix/Library/CloudStorage/OneDrive-TheOhioStateUniversity/2023-2024/Fall/CSPAN/115th_transcript_data/115th_raw_date.csv")


/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_77054/2633198838.py:1: DtypeWarning: Columns (5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,2

In [219]:
df= df[["url", "time", "name", "party-state", "text", "congress", "date"]]

In [220]:
df

,url,time,name,party-state,text,congress,date
0,https://www.c-span.org/video/?420804-1&action=...,0:00:37,NaN,NaN,115th congress of the united states the house...,115,3-Jan-17
1,https://www.c-span.org/video/?420804-1&action=...,0:00:39,NaN,NaN,order. the prayer will be offered by the very ...,115,3-Jan-17
2,https://www.c-span.org/video/?420804-1&action=...,0:01:23,NaN,NaN,give them wisdom so that they may lead the peo...,115,3-Jan-17
3,https://www.c-span.org/video/?420804-1&action=...,0:02:52,NaN,NaN,i pledge allegiance to the flag of the united ...,115,3-Jan-17
4,https://www.c-span.org/video/?420804-1&action=...,0:03:20,NaN,NaN,the representatives elect will record their pr...,115,3-Jan-17
...,...,...,...,...,...,...,...
67195,https://www.c-span.org/video/?456223-1&action=...,0:32:08,NaN,NaN,"thank you, mr. speaker.i yield myself such ti...",115,21-Dec-18
67196,https://www.c-span.org/video/?456223-1&action=...,0:32:11,NaN,NaN,the gentleman is recognized.,115,21-Dec-18
67197,https://www.c-span.org/video/?456223-1&action=...,0:32:13,Jamie Raskin,D-MD,"jamie raskin, d-mdi support s. 3191, the civil...",115,21-Dec-18
67198,https://www.c-span.org/video/?456223-1&action=...,0:33:09,NaN,NaN,the question is will the house suspend the rul...,115,21-Dec-18


In [237]:
df['url'] = df['url'].astype("string")

df['time'] = df['time'].astype("string")
df['name'] = df['name'].astype("string")
df['party-state'] = df['party-state'].astype("string")
df['date'] = df['date'].astype("string")
df['text'] = df['text'].astype("string")
df['congress'] = df['congress'].astype("string")



/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_77054/4211742417.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['url'] = df['url'].astype("string")
/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_77054/4211742417.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = df['time'].astype("string")
/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_77054/4211742417.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [238]:
df.dtypes

url            string[python]
time           string[python]
name           string[python]
party-state    string[python]
text           string[python]
congress       string[python]
date           string[python]
dtype: object

In [239]:
df["text"].iloc[0].lower()

'115th congress  of the united states the house  will come to '

In [244]:
type(df["text"][0])

str

In [251]:
sum(df["text"].isna())

0

In [250]:
df["text"] = df["text"].fillna('')

/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_77054/1807524234.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = df["text"].fillna('')


In [252]:
df["one_minute_bin"] = df["text"].apply(find_string)

/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_77054/2503053443.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["one_minute_bin"] = df["text"].apply(find_string)


In [254]:
one_minute_df = df[df["one_minute_bin"] ==True]
one_minute_df['index_col']= one_minute_df.index

/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_77054/2180051814.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_minute_df['index_col']= one_minute_df.index


In [277]:
df[50:100]

,url,time,name,party-state,text,congress,date,one_minute_bin
50,https://www.c-span.org/video/?420804-1&action=...,2:39:40,<NA>,<NA>,"the speaker pro tempore: without objection, th...",115,3-Jan-17,False
51,https://www.c-span.org/video/?420804-1&action=...,2:39:46,<NA>,<NA>,for what purpose does the gentlelady from the...,115,3-Jan-17,False
52,https://www.c-span.org/video/?420804-1&action=...,2:39:51,<NA>,<NA>,norton: i rise to offer a motion that is at th...,115,3-Jan-17,False
53,https://www.c-span.org/video/?420804-1&action=...,2:40:06,<NA>,<NA>,"anderson, upholding the constitutionality of s...",115,3-Jan-17,False
54,https://www.c-span.org/video/?420804-1&action=...,2:41:00,<NA>,<NA>,clerk: mr. mccarthy of california moves to lay...,115,3-Jan-17,False
55,https://www.c-span.org/video/?420804-1&action=...,2:41:07,<NA>,<NA>,so many as are in favor say aye. those opposed...,115,3-Jan-17,False
56,https://www.c-span.org/video/?420804-1&action=...,2:41:15,<NA>,<NA>,norton: on that i demand the yeas and nays. th...,115,3-Jan-17,False
57,https://www.c-span.org/video/?420804-1&action=...,2:41:22,<NA>,<NA>,"a sufficient number having arisen, the yeas an...",115,3-Jan-17,False
58,https://www.c-span.org/video/?420804-1&action=...,2:41:27,<NA>,<NA>,made possible by the national captioning insti...,115,3-Jan-17,False
59,https://www.c-span.org/video/?420804-1&action=...,3:04:28,<NA>,<NA>,the speaker pro tempore: on this vote the yeas...,115,3-Jan-17,False


In [283]:
df[88:92]

,url,time,name,party-state,text,congress,date,one_minute_bin
88,https://www.c-span.org/video/?420804-1&action=...,3:36:33,<NA>,<NA>,"sanchez: thank you, mr. speaker. i rise in opp...",115,3-Jan-17,False
89,https://www.c-span.org/video/?420804-1&action=...,3:36:41,<NA>,<NA>,changes that truly undermine the very foundati...,115,3-Jan-17,False
90,https://www.c-span.org/video/?420804-1&action=...,3:37:45,<NA>,<NA>,the speaker pro tempore: the gentlewoman from...,115,3-Jan-17,True
91,https://www.c-span.org/video/?420804-1&action=...,3:37:48,<NA>,<NA>,delauro: this rules package sets a disturbing ...,115,3-Jan-17,False


In [272]:
df[67057:67062]

,url,time,name,party-state,text,congress,date,one_minute_bin
67057,https://www.c-span.org/video/?456223-1&action=...,0:02:47,<NA>,<NA>,to address the house for one minute.,115,21-Dec-18,True
67058,https://www.c-span.org/video/?456223-1&action=...,0:02:49,<NA>,<NA>,the gentleman is recognized.,115,21-Dec-18,False
67059,https://www.c-span.org/video/?456223-1&action=...,0:02:51,John James Duncan Jr.,R-TN,"john james duncan jr., r-tntoo many of our lea...",115,21-Dec-18,False
67060,https://www.c-span.org/video/?456223-1&action=...,0:03:48,<NA>,<NA>,for what purpose does the gentleman from west...,115,21-Dec-18,False
67061,https://www.c-span.org/video/?456223-1&action=...,0:03:59,<NA>,<NA>,i ask unanimous consent to address the house ...,115,21-Dec-18,True


In [265]:
pd.isna(df["name"][0])

True

In [266]:
# preininitalize columns of df 
colnames= ["url", "start_time", "end_time", "name", "party_state", "text", 'congress', "date"]
new_df = pd.DataFrame(columns = colnames)

In [268]:
ct=0 
forgotten_indicies= []
for i in range(0, len(one_minute_df)): 
    index= int(one_minute_df.iloc[i].index_col)
    
    
    if pd.isna(df.iloc[index+1]["name"]) == False: #if name of next line is not NA... 
        url = df.iloc[index+1].url 
        start_time =  df.iloc[index+1].time
        name = df.iloc[index+1]["name"]  
        party_state =  df.iloc[index+1]["party-state"]
        congress = 116 
        date= df.iloc[index+1].date
        
        #this should be 1 min speech
        if pd.isna(df.iloc[index+1]["name"]) or pd.isna(df.iloc[index+2]["name"]): 
            pass 
        else: 
            if df.iloc[index+1]["name"] == df.iloc[index+2]["name"]: #if name matches for next two, combine 
                text_of_speech= df.iloc[index+1].text + df.iloc[index+2].text
                end_time = df.iloc[index+3].time 
                ct+=1

            else: 
                text_of_speech= df.iloc[index+1].text
                end_time = df.iloc[index+2].time


            list_i = [url, start_time, end_time, name, party_state,text_of_speech, congress, date]
            new_df.loc[len(new_df)] = list_i
    else: #if name of next is na 
        
        # if it says is recognized and doesn't say false, i.e. won't be caught otherwise 
        if ("is recognized" in df.iloc[index+1].text) and (df.iloc[index+1].one_minute_bin == False): 
            
            #if the name isn't NA for the next one 
            if pd.isna(df.iloc[index+2]["name"]) == False:
                url = df.iloc[index+2].url 
                start_time =  df.iloc[index+2].time
                name = df.iloc[index+2]["name"]  
                party_state =  df.iloc[index+2]["party-state"]
                congress = 116 
                date= df.iloc[index+2].date
                
                #this should be 1 min speech
                if pd.isna(df.iloc[index+1]["name"]) or pd.isna(df.iloc[index+2]["name"]): 
                    pass 
                else: 
                    if df.iloc[index+2]["name"] == df.iloc[index+3]["name"]: #if name matches for next two, combine 
                        text_of_speech= df.iloc[index+2].text + df.iloc[index+3].text
                        end_time = df.iloc[index+4].time 
                        ct+=1
                    else: 
                        text_of_speech= df.iloc[index+2].text
                        end_time = df.iloc[index+3].time
                list_i = [url, start_time, end_time, name, party_state,text_of_speech, congress, date]
                new_df.loc[len(new_df)] = list_i
            
        else: 
            #also appends ones who get caught the next go around, i.e. because one_minute_bin is True 
            forgotten_indicies.append(i)
                
            
    
    #mini_df= df.iloc[index: index+5]


In [269]:
ct

461

In [270]:
new_df

,url,start_time,end_time,name,party_state,text,congress,date
0,https://www.c-span.org/video/?420804-1&action=...,3:39:32,3:39:44,Louise Slaughter,D-NY,"louise slaughter, d-nymr. swalwell: today star...",116,3-Jan-17
1,https://www.c-span.org/video/?420804-1&action=...,3:40:48,3:41:00,Louise Slaughter,D-NY,"louise slaughter, d-nymeans, mr. neal. mr. nea...",116,3-Jan-17
2,https://www.c-span.org/video/?420804-1&action=...,3:39:32,3:39:44,Louise Slaughter,D-NY,"louise slaughter, d-nymr. swalwell: today star...",116,3-Jan-17
3,https://www.c-span.org/video/?420804-1&action=...,3:40:48,3:41:00,Louise Slaughter,D-NY,"louise slaughter, d-nymeans, mr. neal. mr. nea...",116,3-Jan-17
4,https://www.c-span.org/video/?420995-101&actio...,0:02:23,0:03:37,Buddy Carter,R-GA,"buddy carter, r-gajimmy burnson who has served...",116,4-Jan-17
...,...,...,...,...,...,...,...,...
932,https://www.c-span.org/video/?456020-6&action=...,1:38:17,1:40:52,Alcee L. Hastings,D-FL,"alcee l. hastings, d-flmr. speaker, it is with...",116,19-Dec-18
933,https://www.c-span.org/video/?456020-6&action=...,1:59:58,2:01:11,Dan Newhouse,R-WA,"dan newhouse, r-wai rise to honor the life, se...",116,19-Dec-18
934,https://www.c-span.org/video/?456178-1&action=...,0:03:14,2:01:11,Ted Poe,R-TX,"dan newhouse, r-wai rise to honor the life, se...",116,20-Dec-18
935,https://www.c-span.org/video/?456178-2&action=...,2:20:07,2:01:11,Cedric Richmond,D-LA,"dan newhouse, r-wai rise to honor the life, se...",116,20-Dec-18
